## 1. Business Understanding
### 1.1 Overview
To expand its market footprint and diversify its investment portfolio, a company is preparing to enter the aviation sector. This initiative involves acquiring aircrafts to operate within both commercial and private aviation markets. Given the significant risks associated with air travel-ranging from mechanical failure and environmental hazards to pilot error and maintenance practices-it is imperative that the company make data-informed decisions regarding aircraft selection.

### 1.2 Business Problem
The organization currently lacks knowledge about the risk profiles of different aircraft models. Without a clear understanding of historical accident patterns, the company could unknowingly invest in aircraft with poor safety records, resulting in potential financial loss, reputational damage, and regulatory complications. Therefore, leadership has tasked the data team (me) with identifying the **lowest-risk aircraft models** to guide strategic procurement.

### 1.3 Project Objective
The objective of this project is to analyze historical aviation accident data to evaluate and compare the safety performance of different aircraft models. The goal is to translate this analysis into **three clear business recommendations** that will inform the Aviation Division’s purchasing decisions.

This involves:
- Identifying aircraft models with consistently low accident frequencies or severities.
- Understanding trends across aircraft manufacturers, types, and use cases (commercial vs. private).
- Assessing the impact of contributing factors such as pilot error, equipment failure, weather conditions, or operational mismanagement.

### 1.4 Business Goals
- **Minimize Risk**: Recommend aircraft with the lowest historical accident rates to reduce the risk exposure for the business.
- **Support Procurement**: Provide a ranked list or categorical insights on safe aircraft for commercial and private deployment.
- **Enable Strategic Planning**: Use historical data trends to anticipate long-term implications of choosing particular aircraft.

### 1.5 Success Criteria
- Delivery of **three actionable and evidence-based business recommendations** supported by visual insights.
- Development of an **interactive dashboard** that allows business stakeholders to explore aircraft risk profiles.
- A **non-technical presentation** and a **well-documented Jupyter Notebook** that together communicate the methodology, findings, and value of the analysis.

In [164]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
aviation_data = pd.read_csv(r'Data\AviationAccidentDataset\AviationData.csv', encoding='ISO-8859-1')

aviation_data.head()

c:\Users\lenovo\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [165]:
'''IDE- Initial Data Exploration'''

# Finding out how many rows and columns are available
print(f"The dataset contains {aviation_data.shape[0]} rows and {aviation_data.shape[1]} columns\n")

# Find out column names to know if they need standardisation and renaming
print("Column Names:\n", aviation_data.columns, "\n")

The dataset contains 88889 rows and 31 columns

Column Names:
 Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object') 



In [166]:
# Standardize columns for easier readability
aviation_data.columns = (aviation_data.columns.str.strip().str.lower().str.replace(".", "_"))

aviation_data.sample(2) # To check if changes reflect

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
34368,20001206X00606,Accident,CHI94LA063,1994-01-12,"CASSELTON, ND",United States,NaN,NaN,5N8,CASSELTON REGIONAL,...,Instructional,NaN,0.0,0.0,0.0,1.0,VMC,Landing,Probable Cause,13-03-1995
69075,20101011X34733,Accident,ERA11CA017,2010-10-11,"Falls Village, CT",United States,415718N,0732150W,NONE,Private Grass Strip,...,Positioning,Willie Simon,0.0,0.0,0.0,1.0,VMC,NaN,"The pilots failure to abort the takeoff, whic...",25-09-2020


In [167]:
# Get metadata
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                88889 non-null  object 
 1   investigation_type      88889 non-null  object 
 2   accident_number         88889 non-null  object 
 3   event_date              88889 non-null  object 
 4   location                88837 non-null  object 
 5   country                 88663 non-null  object 
 6   latitude                34382 non-null  object 
 7   longitude               34373 non-null  object 
 8   airport_code            50249 non-null  object 
 9   airport_name            52790 non-null  object 
 10  injury_severity         87889 non-null  object 
 11  aircraft_damage         85695 non-null  object 
 12  aircraft_category       32287 non-null  object 
 13  registration_number     87572 non-null  object 
 14  make                    88826 non-null

In [168]:
# Getter a better view of all numeric variables
numeric_cols = aviation_data.select_dtypes(include = 'number')

for col in numeric_cols:
    print(col)

number_of_engines
total_fatal_injuries
total_serious_injuries
total_minor_injuries
total_uninjured


In [169]:
# Get a better view of all categorical variables
categorical_cols = aviation_data.select_dtypes(include = 'object')

for col in categorical_cols:
    print(col)


event_id
investigation_type
accident_number
event_date
location
country
latitude
longitude
airport_code
airport_name
injury_severity
aircraft_damage
aircraft_category
registration_number
make
model
amateur_built
engine_type
far_description
schedule
purpose_of_flight
air_carrier
weather_condition
broad_phase_of_flight
report_status
publication_date


In [170]:
'''Descriptive Analysis'''
# Get Statistical summary
aviation_data.describe(include = "O").T

,count,unique,top,freq
event_id,88889,87951,20001212X19172,3
investigation_type,88889,2,Accident,85015
accident_number,88889,88863,CEN22FA424,2
event_date,88889,14782,1982-05-16,25
location,88837,27758,"ANCHORAGE, AK",434
country,88663,219,United States,82248
latitude,34382,25592,332739N,19
longitude,34373,27156,0112457W,24
airport_code,50249,10375,NONE,1488
airport_name,52790,24871,Private,240


In [171]:
# Check for duplicated values
print("Duplicates:", aviation_data.duplicated().sum())

# Check for nulls and get their percentage to advice on best imputing or dropping criteria
null_counts = aviation_data.isna().sum()
null_percent = (null_counts / len(aviation_data)) * 100
null_summary = pd.DataFrame({'Null Count': null_counts, 'Null %': null_percent.round(2)})

print("\nNull Values Summary:\n", null_summary)

Duplicates: 0

Null Values Summary:
                         Null Count  Null %
event_id                         0    0.00
investigation_type               0    0.00
accident_number                  0    0.00
event_date                       0    0.00
location                        52    0.06
country                        226    0.25
latitude                     54507   61.32
longitude                    54516   61.33
airport_code                 38640   43.47
airport_name                 36099   40.61
injury_severity               1000    1.12
aircraft_damage               3194    3.59
aircraft_category            56602   63.68
registration_number           1317    1.48
make                            63    0.07
model                           92    0.10
amateur_built                  102    0.11
number_of_engines             6084    6.84
engine_type                   7077    7.96
far_description              56866   63.97
schedule                     76307   85.85
purpose_of_flight

## 2. Data Understanding

### 2.1 Data Source

The dataset used in this analysis is sourced from the **National Transportation Safety Board (NTSB)** and contains records of civil aviation accidents and selected incidents from **1962 to 2023**. The data covers both U.S.-based incidents and those that occurred in international waters involving U.S.-registered aircraft.

- **File Type**: CSV
- **Size**: Dataset has 88889 rows and 31 columns
- **Period Covered**: 1962–2023
- **Scope**: Includes variables on aircraft make/model, accident severity, causes, weather conditions, flight purpose, location, and fatalities/injuries.

### 2.2 Data Structure

Key columns in the dataset include:

| Column Name             | Description                                                                 |
|-------------------------|-----------------------------------------------------------------------------|
| `make`                  | Manufacturer of the aircraft                                                |
| `model`                 | Model of the aircraft                                                       |
| `aircraft_damage`       | Severity of aircraft damage (e.g., Destroyed, Substantial, Minor)           |
| `injury_severity`       | Level of injury/fatalities (e.g., Fatal, Non-Fatal, None)                   |
| `purpose_of_flight`     | Reason for flight (e.g., Personal, Business, Commercial, Instructional)     |
| `broad_phase_of_flight` | Flight phase during which the incident occurred (e.g., Takeoff, Landing)    |
| `weather_condition`     | Weather during the incident (e.g., VMC - Visual Meteorological Conditions)  |
| `engine_type`           | The type of engine of the aircraft (e.g., Reciprocating, Turbo Fan, Turbo Jet, etc.) |
| `amateur_built`         | Indicates whether the aircraft was amateur/home-built (`Yes` or `No`)       |
| `schedule`              | Indicates if the flight was scheduled or not (`SCH` for scheduled, `NSCH` for non-scheduled) |


### 2.3 Initial Observations

- **Duplicates**: Zero (0)
- **Missing/Null Values**: Missing data is prevalent across several columns- some like 'schedule' have 85% of it having missing values.
- The dataset is largely comprised of categorical variables with the exception of 'number_of_engines', 'total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries' and 'total_uninjured' which are numeric variables. These may require normalisation.
- The dataset spans **over six decades**, making time-based trend analysis highly feasible.
- The columns **'make'** and **'model'** are critical for this analysis since they relate directly to the business question on **aircraft risk assessment**.

### 2.4 Data Quality Issues

- **Missing Values**: Several fields contain nulls or blank strings. These must be investigated for relevance and either imputed, ignored, or used as-is depending on the column.
- **Inconsistent Labeling**: Categorical values such as 'aircraft_damage' or 'purpose_of_flight' may be inconsistent (e.g., "business" vs "Business") and will require standardization.
- **Outliers**: Check if outliers exist in fields like 'event_date', 'pubication_date' (among others) and must be handled with care.

## 3. Next Steps

- Perform data cleaning and preprocessing.
- Explore distributions of key fields (e.g., damage, injury, make/model frequency).
- Diagnostic analysis, Descriptive analysis and Prescriptive analysis which will include uni-variate, bi-variate and multi-variate analysis. This will be done by creating visualisation and interpreting them.

### 3.1 Data Cleaning
- Impute missing data with appropriate imputation method- median, mode.
- Standardise entries that need standardisation.
- Check for outliers in relevant columns and deal with them appropriately. 
- Feature engineering
- Drop columns I will not need/use for the analysis.

In [ ]:
'''DATA CLEANING'''

''' 1. make ''' 

# Find Null count in 'make' column
print("Null Values in 'make':", aviation_data['make'].isna().sum())

# Get unique values in 'make' column
print("\nUnique values in 'make':\n", aviation_data['make'].unique())

# Standardise 'make' column fields
aviation_data['make'] = aviation_data['make'].str.strip().str.title()

# See the 20 most common makes
print("\nThe 15 Most Common Aicraft Makes are:\n", aviation_data['make'].value_counts().head(15))

Null Values in 'make': 63

Unique values in 'make':
 ['Stinson' 'Piper' 'Cessna' ... 'JAMES R DERNOVSEK' 'ORLICAN S R O'
 'ROYSE RALPH L']

The 15 Most Common Aicraft Makes are:
 Cessna               27149
Piper                14870
Beech                 5372
Boeing                2745
Bell                  2722
Mooney                1334
Robinson              1230
Grumman               1172
Bellanca              1045
Hughes                 932
Schweizer              773
Air Tractor            691
Aeronca                636
Mcdonnell Douglas      608
Maule                  589
Name: make, dtype: int64


In [ ]:
''' Deal with missing values in make column '''

# Considering that they are only 0.07% of the entire data, filling them with mode (because it is categorical) is the best option
aviation_data['make'].fillna(aviation_data['make'].mode()[0], inplace = True)

print("Null Values in 'make' after imputation:", aviation_data['make'].isna().sum())

Null Values in 'make' after imputation: 0


In [ ]:
''' 2. model '''

# Find count of null values in model column
print("\nNull Values in 'model':", aviation_data['location'].isna().sum())

# Find all unique values in model column
print("\nUnique 'model':\n", aviation_data['model'].unique()) 

# Get the top 20 aircraft models
print("\nTop 15 Aircraft Models:\n", aviation_data['model'].value_counts().head(15))



Null Values in 'model': 52

Unique 'model':
 ['108-3' 'PA24-180' '172M' ... 'ROTORWAY EXEC 162-F' 'KITFOX S5'
 'M-8 EAGLE']

Top 15 Aircraft Models:
 152          2367
172          1756
172N         1164
PA-28-140     932
150           829
172M          798
172P          689
182           659
180           622
150M          585
PA-18         581
PA-18-150     578
PA-28-180     572
PA-28-161     569
PA-28-181     532
Name: model, dtype: int64


In [ ]:
''' Deal with missing values in 'model' column '''

# Considering that they are only 0.10% of the entire data, filling them with mode (because it is categorical) is the best option
aviation_data['model'].fillna(aviation_data['model'].mode()[0], inplace = True)

print("Null Values in 'model' after imputation:", aviation_data['model'].isna().sum())

Null Values in 'model' after imputation: 0


In [ ]:
'''3. injurity_severity'''

print("Null values in 'injury_severity' are:", aviation_data['injury_severity'].isna().sum())
print("\nUnique Values in 'injury_severity':\n", aviation_data['injury_severity'].unique())

Null values in 'injury_severity' are: 1000

Unique Values in 'injury_severity':
 ['Fatal(2)' 'Fatal(4)' 'Fatal(3)' 'Fatal(1)' 'Non-Fatal' 'Incident'
 'Fatal(8)' 'Fatal(78)' 'Fatal(7)' 'Fatal(6)' 'Fatal(5)' 'Fatal(153)'
 'Fatal(12)' 'Fatal(14)' 'Fatal(23)' 'Fatal(10)' 'Fatal(11)' 'Fatal(9)'
 'Fatal(17)' 'Fatal(13)' 'Fatal(29)' 'Fatal(70)' 'Unavailable'
 'Fatal(135)' 'Fatal(31)' 'Fatal(256)' 'Fatal(25)' 'Fatal(82)'
 'Fatal(156)' 'Fatal(28)' 'Fatal(18)' 'Fatal(43)' 'Fatal(15)' 'Fatal(270)'
 'Fatal(144)' 'Fatal(174)' 'Fatal(111)' 'Fatal(131)' 'Fatal(20)'
 'Fatal(73)' 'Fatal(27)' 'Fatal(34)' 'Fatal(87)' 'Fatal(30)' 'Fatal(16)'
 'Fatal(47)' 'Fatal(56)' 'Fatal(37)' 'Fatal(132)' 'Fatal(68)' 'Fatal(54)'
 'Fatal(52)' 'Fatal(65)' 'Fatal(72)' 'Fatal(160)' 'Fatal(189)'
 'Fatal(123)' 'Fatal(33)' 'Fatal(110)' 'Fatal(230)' 'Fatal(97)'
 'Fatal(349)' 'Fatal(125)' 'Fatal(35)' 'Fatal(228)' 'Fatal(75)'
 'Fatal(104)' 'Fatal(229)' 'Fatal(80)' 'Fatal(217)' 'Fatal(169)'
 'Fatal(88)' 'Fatal(19)' 'Fatal(60)' 'Fa

In [ ]:
# Standardise injury_severity fields
aviation_data['injury_severity'].str.strip() # remove whitespaces

# Standardise 'Fatal(...)' as just 'Fatal'
aviation_data['injury_severity'] = aviation_data['injury_severity'].str.replace(r'Fatal\(\d+\)', 'Fatal', regex = True)
print("Unique Values in 'injury_severity':\n", aviation_data['injury_severity'].unique())

aviation_data['injury_severity'].value_counts()

Unique Values in 'injury_severity':
 ['Fatal' 'Non-Fatal' 'Incident' 'Unavailable' nan 'Minor' 'Serious']


Non-Fatal      67357
Fatal          17826
Incident        2219
Minor            218
Serious          173
Unavailable       96
Name: injury_severity, dtype: int64

In [180]:
# Make 5 clean categories from injury_severity: Fatal, Serious, Minor, No Injury and Unknown
# Mapping dictionary
severity_map = {
    'Fatal': 'Fatal',
    'Serious': 'Serious',
    'Minor': 'Minor',
    'Non-Fatal': 'No Injury',
    'Incident': 'No Injury',
    'Unavailable': 'Unknown'
}

# Apply mapping
aviation_data['injury_severity'] = aviation_data['injury_severity'].replace(severity_map)
aviation_data['injury_severity'].value_counts()

No Injury    69576
Fatal        17826
Minor          218
Serious        173
Unknown         96
Name: injury_severity, dtype: int64

In [ ]:
'''Deaking with Nulls in 'injury_severity' column'''

# Fill NaNs with 'Unknown' to maintain transparency
# Using mode would introduce bias
aviation_data['injury_severity'] = aviation_data['injury_severity'].fillna('Unknown')

print(aviation_data['injury_severity'].value_counts()) # see new values of 'Unknown'

print("\nNull values in 'injury_severity' after imputation:", aviation_data['injury_severity'].isna().sum())

No Injury    69576
Fatal        17826
Unknown       1096
Minor          218
Serious        173
Name: injury_severity, dtype: int64

Null values in 'injury_severity' after imputation: 0


In [ ]:
''' 4. aircraft_damage '''

# Check how many nulls are in aircraft_damage
print("Null Values in 'aircraft_damage':", aviation_data['aircraft_damage'].isna().sum())

# Check unique values to get a better understanding of the column
print("\nUnique Values in 'aircraft_damage':", aviation_data['aircraft_damage'].unique())

# Check value counts for the unique values to inform on the best imputation method
print("\nValue Counts of unique counts in 'aircraft_damage':\n", aviation_data['aircraft_damage'].value_counts())

Null Values in 'aircraft_damage': 3194

Unique Values in 'aircraft_damage': ['Destroyed' 'Substantial' 'Minor' nan 'Unknown']

Value Counts of unique counts in 'aircraft_damage':
 Substantial    64148
Destroyed      18623
Minor           2805
Unknown          119
Name: aircraft_damage, dtype: int64


In [190]:
'''Dealing will null values in 'aircraft_damage' column'''

# Fill nan with 'Unknown' to maintain transparency 
# Using Mode would introduce bias
aviation_data['aircraft_damage'] = aviation_data['aircraft_damage'].fillna('Unknown')

print(aviation_data['aircraft_damage'].value_counts()) # see new values of 'Unknown'

print("\nNull values in 'aircraft_damage' after imputation:", aviation_data['aircraft_damage'].isna().sum())

Substantial    64148
Destroyed      18623
Unknown         3313
Minor           2805
Name: aircraft_damage, dtype: int64

Null values in 'aircraft_damage' after imputation: 0


In [ ]:
'''5. weather_condition '''
print("Null Values in 'weather_condition':\n", aviation_data['weather_condition


SyntaxError: EOL while scanning string literal (<ipython-input-191-9a8a41e7c471>, line 2)